In [5]:
import matplotlib.pyplot as plt
import statsmodels.api as sm
from SpiPy.Models.SAR import *
from DTO.Database import *
from numpy import load

In [3]:
db_manager = HLDatabase(bWorkLaptop=False)
pollution = db_manager.get_table(table_name='Train-Pollution')
# pollution.drop(labels='Da', axis=1, inplace=True)
mSpatial = db_manager.get_table(table_name='Train-Weight Matrix')
mSpatial.drop(labels='index', axis=1, inplace=True)

tWind = load(r"../DTO/train_tWind.npy")
tZ = load(r"../DTO/train_tY.npy")

In [3]:
N = len(pollution.columns)
initial_params = [0.19, -0.20, 0.31, -0.43, 0.50, -0.66, 0.44]
initial_params += list(pollution.var().values)
initial_params += list(pollution.mean().values)
initial_params += [0.21]
initial_params += [-0.8]
initial_params += [0.33]
initial_params += [2.00]
initial_params += [5.05]

In [4]:
bounds = [(-1, 1)] * N               # Phi
bounds += [(1, 1000)] * N            # Sigma
bounds += [(0, 1000)] * N            # Mu
bounds += [(-100, 100)]              # Alpha
bounds += [(-100, 100)]              # Rho
bounds += [(0, 1)]                   # Zeta
bounds += [(-100, 100)] * 2          # Beta and Gamma

In [ ]:
optimizer = QMLEOptimizer(
    initial_params=initial_params,
    weight_matrix=mSpatial,
    wind_tensor=tWind,
    exog=pollution,
    bounds=bounds,
)

optimizer.fit()
best_params = optimizer.get_best_params()

print("Best Parameters:")
print("Alpha:",      best_params[-5])
print("Rho:",        best_params[-4])
print("Zeta:",       best_params[-3])
print("Beta:",       best_params[-2])
print("Gamma:",      best_params[-1])
print("Phi:",        best_params[:N])
print("Sigma:",   best_params[N:2*N])
print("Mu:",    best_params[2*N:3*N])